# Deltares Software Days 2024: A simple hydrolib-core demo

Hydrolib-core is a pip-installable python package. 
It consists of wrappers around the D-HYDRO model files.

Classes in Hydrolib-core are hierarchically organized by file type, and can be accessed via `hydrolib.core.io.<filetype>.models`.

File objects in Hydrolib-core reflect the same structure as the file contents. 

In this demo, we will load a Flow FM model, make some changes to the model and save it again.

💡 **Tips for working in the Jupyter Notebook:**
  * __Tab__: Auto-complete on code
  * __Ctrl__+__Enter__: Run cell 
  * __Shift__+__Enter__: Run cell and move to next cell 

## 0. Importing the modules and setting the path variables

In [ ]:
from hydrolib.core.dflowfm import (
    FMModel,
    Lateral,
    Weir, FlowDirection
)
from pathlib import Path

root = Path.cwd()
demo_data_folder = root / "data"
export_folder = root / "export"

## 1. Loading a Flow FM model

In [ ]:
mdu_file_path = demo_data_folder / "FlowFM.mdu"
model = FMModel(filepath=mdu_file_path)

print(f"Loaded the model from {model.filepath}")

## 2. Inspecting the model

We can see the hierarchy tree of the model by calling the following function:

In [ ]:
model.show_tree()

We can also inspect sub-parts of the model, like the structures.

Note that a model can have multiple structure files, but this model only has one, so we will get the first and only one.

In [ ]:
assert model.geometry.structurefile is not None
structure_file = model.geometry.structurefile[0]

# Show the list of all the structures
print(f"Number of structures: {len(structure_file.structure)}")
print(structure_file.structure)

## 3. Adjusting the model

### a. Creating a new weir and adding it to the structures**

In [ ]:
# Create a new weir.
weir = Weir(
    id="DV_1471", 
    branchId="458", 
    chainage=106.277467, 
    allowedFlowDir=FlowDirection.both, 
    crestLevel=0.400, 
    crestWidth=40.000, 
    corrCoeff=1.000, 
    useVelocityHeight=True
)

# It is also possible to create objects with a dictionary.
weir_data = {
    "id": "DV_1471", 
    "branchId": "458", 
    "chainage": 106.277467, 
    "allowedFlowDir": FlowDirection.both, 
    "crestLevel": 0.400, 
    "crestWidth": 40.000, 
    "corrCoeff": 1.000, 
    "useVelocityHeight": True
}
weir = Weir(**weir_data)

# Add the weir to the model
structure_file.structure.append(weir)

print(f"Number of structures: {len(structure_file.structure)}")


### b. Adjusting some parameters of the model

In [ ]:
model.physics.backgroundsalinity = 30 # [ppt]
model.physics.backgroundwatertemperature = 6 # [°C]
model.time.dtuser = 900 # [s]

Hydrolib-core offers **instant validation** of objects that are created, either by loading them from file or by manually creating them. This makes it easier for users to create valid models.

A lateral, like many other objects, should have a valid location specification, specified by:
* A node id,
* A branch id with a chainage, or
* Coordinates

Let's create an invalid Lateral. We "forget" to specify a chainage.

In [ ]:
try:
    lateral = Lateral(
        id="lateral_1",
        name="lateral_1",
        branchid="branch_a",
        discharge="realtime"
    )

except ValueError as error:
    print(error)

Now let's create a valid lateral.

In [ ]:
lateral = Lateral(
    id="lateral_1",
    name="lateral_1",
    branchid="branch_a",
    chainage=100,
    discharge="realtime"
)

print("Lateral with branch id and chainage is correct!")

## 5. Saving the model
If we save the model now, it will overwrite the current model files.
So let's save it in a different location.

In [ ]:
print(f"The original model is located at {model.filepath}")

save_mdu_file_path = export_folder / "save_model" / "FlowFM.mdu"
model.filepath = save_mdu_file_path

# Set recurse to True. If it is False, only the MDU file will be saved.
model.save(recurse=True)

print(f"The saved model is located at {model.filepath}")

## 6. Saving only a sub-part of the model
It is also possible to save individual child model files, such as the cross section definition file.

In [ ]:
crossdef_file = model.geometry.crossdeffile
assert crossdef_file is not None

crossdef_file.filepath = export_folder / "save_crossdef" / "crsdef.ini"
crossdef_file.save()

print(f"The saved cross section definition model is located at {crossdef_file.filepath}")